# Train a model

With the knowledge of some Spark and PySpark basics, one can move on to machine learning utilities: Spark ML (*) 

Here, we take a simple dataset and fit a regressor. This does not claim to be a complete data science process. More important is to get to know the analogies and differences to known frameworks such as scikit-learn.

(*) Often, one hears the term *MLLib*. This usually refers to the *ML* library by Spark. Strictly speaking, these are two different things. Spark MLlib is the older of the two and is applied directly to RDDs. Spark ML, on the other hand, is built on top of DataFrames and provides a higher-level API that abstracts away some of the low-level details of building and tuning machine learning models.

In [ ]:
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev

In [ ]:
spark_session: SparkSession = SparkSession.builder.master("local").appName("Local").getOrCreate()

In [ ]:
sc = spark_session.sparkContext

## Load data

Iris, penguins, AirBnBs, Wines etc. You already know them by heart?   

Well, then let's take a look at a more business specific problem.

In [ ]:
df = spark_session.read.csv("data/mocked_customer_data.csv", inferSchema=True, header=True)

## EDA

Performing EDA using PySpark has its pros and cons. On the one hand, PySpark supports various statistical functions that can help you calculate summary statistics, identify outliers, and explore relationships between variables. This can be particularly useful when dealing with very large datasets that cannot be easily processed using other tools.

On the other hand, PySpark does not support plotting and visualization, which can be an important part of the EDA process. 

In [ ]:
df.printSchema()

In [ ]:
df.show(2)

In [ ]:
df.show(2, vertical=True)

In [ ]:
spark_session.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
df

In [ ]:
df_mean = df.select([mean(c) for c in df.columns])

In [ ]:
df_mean.show()

In [ ]:
df_std = df.select([stddev(c) for c in df.columns])

In [ ]:
df_std.show()

## Build a model...

Let's build a simple classification model which aims to predict the *target*.

But first, let's clarify [terminology](https://spark.apache.org/docs/latest/ml-pipeline.html#example-estimator-transformer-and-param) for a moment:

* We already know **DataFrames**.
* **Transformers** can transform one DataFrame to another DataFrame. An ML Transformer transforms a DataFrame with features to DataFrame with predictions.
* **Estimators** create Transformers via Fitting on a DataFrame.
* A **Pipeline** chains Transformers and Estimators to create a Pipeline workflow. But they are not covered here.

In [ ]:
FEATURES = ["n1", "n2", "n3", "n4_1", "n4_2", "n4_3"]
TARGET = "target"

Most machine learning algorithms in Spark require input data to be in vector format. PySpark's VectorAssembler is a utility that allows you to combine multiple columns of a PySpark DataFrame into a single vector column. The resulting vector column is then used as an input to machine learning algorithms in PySpark.

In [ ]:
assembler = VectorAssembler(
    inputCols=FEATURES,
    outputCol="features_vec"
)
df_prep = assembler.transform(df).select("features_vec", TARGET)

In [ ]:
df_prep.show(2)

In [ ]:
df_prep.collect()[0]

In [ ]:
df_train, df_test = df_prep.randomSplit([0.8, 0.2], seed=42)

Allright. Let's fit the baseline model.

The syntax is very similar to sklearn. But, as mentioned above, there is a distingtion between Estimators and Transformers. So, the fitting process returns a Transformer object (in contrast to sklearn, where the object fitted object is able to make predictions directly).

In [ ]:
lr = LogisticRegression(labelCol=TARGET, featuresCol="features_vec")

In [ ]:
model = lr.fit(df_train)

The question whether and how the training process can now be parallelized depends on the model architecture. A knn clustering, for example, can be easily parallelized. An xgboost method, on the other hand, is somewhat more complex. Since the training process is sequential, only the individual steps can be parallelized here. The way in which the calculation is optimized differs in each case. This also applies to the inference.

In [ ]:
df_pred = model.transform(df_test)

## ... test it ...

In [ ]:
df_pred.show(3)

Let's test the goodness of fit via 
* Accuarcy
* F1 Score 
* Area under the curve

Note that neither BinaryClassificationEvaluator nor MulticlassClassificationEvaluator can calculate all metrics on their own. We need to use both.

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol=TARGET, 
                                              predictionCol="prediction")

accuracy = evaluator.evaluate(df_pred, {evaluator.metricName: 'accuracy'})
f1 = evaluator.evaluate(df_pred, {evaluator.metricName: 'f1'})

print(f"accuracy: {accuracy:.2f}")
print(f"f1: {f1:.2f}")

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol=TARGET, 
                                          rawPredictionCol='rawPrediction')

auc = evaluator.evaluate(df_pred, {evaluator.metricName: 'areaUnderROC'})

print(f"AUC: {auc:.2f}")

## ... and persist it.

So here we are. We've got a model in our hands. Hooray.

For sure, we don't want to train a model again and again but persist it. The Transformer object comes with an built in function to do so. One can persist the model on localhost or any blob storage. 

In [ ]:
import os

In [ ]:
path = "model/my_model"

In [ ]:
if os.path.exists(path):
    os.system(f'rm -r {path}')

In [ ]:
model.save(path)

Then, in any production environment, one can load the model to work with it. Note that this could also happen in Scala or Java directly, because the serialization was not done via pickle but is language-agnostic.
Again, one faces some unintuitive syntax, because the ```load()``` method is implemented in a Model class.

In [ ]:
# model_loaded = LogisticRegressionModel.load("model/my_model")

## Chain in a pipeline

There is also the possibility to capture these two steps directly in one object. Pipelines form a construct in which several Transformers and Estimators can be chained together. 

In [ ]:
assembler = VectorAssembler(
    inputCols=FEATURES,
    outputCol="features_vec"
)

lr = LogisticRegression(labelCol=TARGET, featuresCol="features_vec")

pipeline = Pipeline(stages=[assembler, lr])

In this case, the train-test-split has to be applied directly on the original DataFrame. 

In [ ]:
df_train_direct, df_test_direct = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
model = pipeline.fit(df_train_direct)

In [ ]:
df_pred = model.transform(df_test_direct)

In [ ]:
df_pred.show(3)

## Optimization

Note that the example above is for education purposes. In real world problems, one might try out different hyperparameter setups as well as different models to reach the best one. Also, cross validation should be applied if possible to acchieve more stable results.

In [ ]:
assembler = VectorAssembler(
    inputCols=FEATURES,
    outputCol="features_vec"
)

lr = LogisticRegression(labelCol=TARGET, featuresCol="features_vec")

evaluator = BinaryClassificationEvaluator(labelCol=TARGET, 
                                          rawPredictionCol='rawPrediction',
                                          metricName="areaUnderROC")

pipeline = Pipeline(stages=[assembler, lr])

parameter_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=parameter_grid,
    evaluator=evaluator,
    numFolds=2
)

cross_val_model = crossval.fit(df_train_direct)

In [ ]:
DEEP_DIVE = False

In [ ]:
if DEEP_DIVE: 
    print(cross_val_model.bestModel.stages[1].extractParamMap())

In [ ]:
if DEEP_DIVE:
    print(cross_val_model.avgMetrics)

## Challenges in a fitting process.

Real data sets usually require some pre-processing before fitting is possible.

* For instance, not every ML model is able to handle missing values. One needs to drop oder impute missing values first. 
* Also, some models are sensitive to different scales in the expressions of numerical values, so one needs to standardize.
* Speaking about numerical values, for many models categorical data have to be encoded first. 

For those cases, there exist Transformers as well. One should integrate them in the modelling process, best directly into a pipeline.

## Wrap-Up 

* Spark ML is a powerful tool to train ML models in a distributed way.
* The syntax is similar to sklearn, but there are some differences.
* Transformer and Estimator are the two main components in a fitting process. Transformers transform data, Estimators fit models.
* The feature values must first be transferred to a so-called feature vector.
* The fitting process is parallelized, but the degree of parallelization depends on the model architecture.
* The resulting model can be persisted and loaded again in any production environment.
* Pipelines are a convenient way to chain several Transformers and Estimators together.
* Once again, the principle of lazy behavior applies. The fitting process is only executed when the ```fit()``` method is called.

## Congrats!

You dived into ML via Pyspark.  
🚀🚀🚀

Again, stop the local spark session.

Then, let's move to the cloud! 

In [ ]:
spark_session.stop()